In [1]:
import datetime as dt
from collections import OrderedDict
from pathlib import Path
import sys
import os

from dotenv import load_dotenv, find_dotenv

ROOT = Path('../')
DATA_DIR = ROOT/'tests'/'data'
OUT_DIR = ROOT/'output'

sys.path.append(str(ROOT))

%load_ext autoreload
%autoreload 2

import countdowner as cd

load_dotenv(find_dotenv())


True

In [2]:
# Read a watchlist

w = cd.read_watchlist(DATA_DIR/'watchlist.csv')
w


{'email_addresses': ['brainbummer@mailinator.com', 'rhymedude@mailinator.com'],
 'products':   product_description stock_code
 0         Brazil nuts     291156
 1        GB chocolate      32467
 2              cheese     281739}

In [3]:
# Get and parse a product

code = w['products']['stock_code'].iat[0]
r = cd.get_product(32467)
cd.parse_product(r)

{'stock_code': '32467',
 'name': 'Green & Blacks Chocolate Block Dark 85% Cocoa',
 'description': "Passionately crafted by bringing together the finest ingredients, green & black's offers a distinctively delicious, smooth chocolate experience. Green symbolises our commitment to ethically sourced cocoa, & black stands for the high quality and delicious taste of our chocolate, for pure chocolate pleasure. Green & black's organic 85% dark chocolate is made with a generous measure of madagascan vanilla to balance the bitter notes of the cocoa, creating chocolate that is intense and smooth.",
 'size': '90g',
 'unit_price_($)': 4.22,
 'unit_size': '100G',
 'sale_price_($)': 3.8,
 'normal_price_($)': 3.8,
 'discount_(%)': 0.0,
 'datetime': '2020-12-09T16:33:49'}

In [4]:
# Get and parse many products

codes = w['products']['stock_code'].values
%time f = cd.collect_products(codes)
f

CPU times: user 64.9 ms, sys: 457 µs, total: 65.4 ms
Wall time: 1.07 s


,stock_code,name,description,size,unit_price_($),unit_size,sale_price_($),normal_price_($),discount_(%),datetime
0,291156,Macro Brazil Nuts,None,250g,3.00,100G,7.5,7.5,0.0,2020-12-09 16:33:50
1,32467,Green & Blacks Chocolate Block Dark 85% Cocoa,Passionately crafted by bringing together the ...,90g,4.22,100G,3.8,3.8,0.0,2020-12-09 16:33:50
2,281739,Mainland Cheese Block Organic Cheddar,Mainland organic cheddar is a mild cheddar che...,500g,32.00,1KG,16.0,16.0,0.0,2020-12-09 16:33:50


In [5]:
# Filter sales

cd.filter_sales(f)

,name,sale_price_($),normal_price_($),discount_(%)


In [6]:
# Email sales
print(f"Emailing to {w['email_addresses']}")

username = os.getenv('GMAIL_USERNAME')  # Replace with your GMail username
password = os.getenv('GMAIL_PASSWORD')  # Replace with your GMail password
cd.email(f, w['email_addresses'], "Hello from Countdowner", username, password)

Emailing to ['brainbummer@mailinator.com', 'rhymedude@mailinator.com']


SSLError: [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1123)

In [ ]:
# Get products in bulk

cd.run_pipeline(DATA_DIR/'watchlist.csv')

In [ ]:
# Run pipeline again: read watchlist, collect products, write products to file, 
# filter sales, and email results

cd.run_pipeline(DATA_DIR/'watchlist.csv', out_path=OUT_DIR/'products.csv')
%less {OUT_DIR/'products.csv'}